In [1]:
import os
import shutil
import sys

import ee
import geemap
import rasterio
import cv2
from PIL import Image

from osgeo import gdal

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import date
from datetime import timedelta

import ast

In [ ]:
# note: have to initialize and authenticate gee before running image downloader
def image_downloader(place_coords, start_date, end_date):
    
    # input given as 'MM/DD/YYY'
    start = datetime.strptime(start_date, '%m/%d/%Y')
    end = datetime.strptime(end_date, '%m/%d/%Y')

    # create folders if not already created
    if not os.path.exists('gee_data'):
        os.mkdir('gee_data')
    if not os.path.exists('gee_data/full_img'):
        os.mkdir('gee_data/full_img')
        
    bbox = place_coords
    
    region = ee.Geometry.Polygon(bbox)

    collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(start_date,end_date).filterBounds(region)
    collection_list = collection.toList(collection.size())
    dates = geemap.image_dates(collection, date_format='YYYY-MM-dd').getInfo()
        
    for i, date in enumerate(dates[:]):
        
        image = ee.Image(collection_list.get(i)).select('VV')

        try:
            geemap.ee_export_image(image, filename = "gee_data/full_img/{}.tif".format(date), region = region)
        except Exception as e:
            print(e)

    print('Successfully Downloaded')
    return

In [ ]:
def image_splitter(img_fp):
    """
    Takes the image filepath returns m x n array of the subimages
    """
#     img = gdal.Open(img_fp)
#     img_array = img.GetRasterBand(1).ReadAsArray()
    with rasterio.open(img_fp) as src:
        img = src.read()[0]
    
    img_height, img_width = img_array.shape
    # get next biggest multiple of 800
    new_height = img_height + (800 - img_height % 800)
    new_width = img_width + (800 - img_width % 800)
    
    reshaped = cv2.resize(img_array, (new_height, new_width))
    
    # for some reason have to rescale or else when saving image it will be dark
    rescaled = a_scaled = 255*(reshaped-reshaped.min())/(reshaped.max()-reshaped.min())
    
    # split image into subimages
        # will return array [m, n, 800, 800] where there are an m x n number of images with size 800x800 
    split = reshape_split(rescaled, kernel_size=(800,800))
    return split

In [ ]:
def inshore_offshore_classifier(img):
    """
    Takes in an image and classifies it as either offshore(1) or inshore(0)
    """
    # load in predictor
    clf = joblib.load("inshore_offshore_clf_model.pkl")
    
    img_vals = np.copy(img)
    img_50 = np.percentile(img_vals,50)
    img_80 = np.percentile(img_vals,80)
    img_90 = np.percentile(img_vals,90)
    img_30 = np.percentile(img_vals,30)
    
    features = np.array(img_50, img_80, img_90, img_30)
    return clf.predict(features)